# Long-Range Person Detection - YOLOv11s with SAHI

This notebook trains YOLOv11s on the VisDrone dataset **filtered for persons only**,
and includes SAHI (Slicing Aided Hyper Inference) for enhanced small object detection.
Optimized for Google Colab T4 GPU.

In [ ]:
# 1. Clone the Repository
!git clone https://github.com/haroon0x/y11-rfdetr.git
%cd y11-rfdetr

In [ ]:
# 2. Install Dependencies
!pip install -r requirements.txt
!pip install ultralytics
!pip install sahi  # SAHI for sliced inference

In [ ]:
# 3. Verify Environment
import torch
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# 4. Prepare VisDrone Dataset (Filter for Persons Only)
!python scripts/prepare_visdrone.py

In [ ]:
# 5. Run Training Pipeline
# Using YOLOv11s (Small) - optimized for Jetson Nano deployment
!python scripts/train_pipeline.py --epochs 50 --batch 8 --imgsz 960 --workers 2 --model yolo11s.pt

In [ ]:
# 6. Display Training Results
import os
from IPython.display import Image, display

base_dir = "yolo11m_training_runs"
step_dir = "step1_visdrone_person"
results_path = os.path.join(base_dir, step_dir, "results.png")
confusion_matrix_path = os.path.join(base_dir, step_dir, "confusion_matrix.png")

if os.path.exists(results_path):
    print("Training Results:")
    display(Image(filename=results_path))
if os.path.exists(confusion_matrix_path):
    print("Confusion Matrix:")
    display(Image(filename=confusion_matrix_path))

In [ ]:
# 7. Benchmark Model
!yolo benchmark model=yolo11m_training_runs/step1_visdrone_person/weights/best.pt data=data/visdrone_person/data.yaml imgsz=640 half=True device=0

In [ ]:
# 8. Run SAHI Inference (Example)
# SAHI improves detection of small/distant persons by slicing the image
!python scripts/inference_sahi.py --model yolo11m_training_runs/step1_visdrone_person/weights/best.pt --source data/visdrone_person/images/val/

In [ ]:
# 9. Download Results
!zip -r training_results.zip yolo11m_training_runs/step1_visdrone_person/ runs/sahi_inference/
from google.colab import files
files.download('training_results.zip')